In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic-dl/train.csv
/kaggle/input/titanic-dl/test.csv


In [2]:
import numpy as np
import pandas as pd
import torch
import warnings 
warnings.filterwarnings('ignore')

In [3]:
input_dim = 4
hidden_dim = 6
output_dim = 1

In [4]:
EPOCHS = 10

In [5]:
X_data = []
X_test_data = []
Y_data = []

In [6]:
df_train = pd.read_csv('/kaggle/input/titanic-dl/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic-dl/test.csv')

In [7]:
# 학습 데이터 준비
def data():
    global X_data, X_test_data, Y_data
    
    Y = df_train["Survived"]
    
    df_train.loc[df_train['Sex'] == 'male', 'Sex'] = 0
    df_train.loc[df_train['Sex'] == 'female', 'Sex'] = 1
    df_train['Sex'] = pd.to_numeric(df_train['Sex'])
    
    df_test.loc[df_test['Sex'] == 'male', 'Sex'] = 0
    df_test.loc[df_test['Sex'] == 'female', 'Sex'] = 1
    df_test['Sex'] = pd.to_numeric(df_test['Sex'])
    
    features = ['Pclass', 'Sex', 'SibSp', 'Parch']
    X = pd.get_dummies(df_train[features])
    X_test = pd.get_dummies(df_test[features])
    
    X_data = torch.from_numpy(X.to_numpy()).float()
    X_test_data = torch.from_numpy(X_test.to_numpy()).float()
    Y_data = torch.from_numpy(Y.to_numpy()).float()

In [8]:
class TitanicModel(torch.nn.Module):
    def __init__(self):
        super(TitanicModel, self).__init__()
        
        self.linear1 = torch.nn.Linear(input_dim, hidden_dim)
        self.ReLU = torch.nn.ReLU()
        self.linear3 = torch.nn.Linear(hidden_dim, output_dim)
        
    def forward(self, x):
        x = self.linear1(x)
        x = self.ReLU(x)
        x = self.linear3(x)
        return x

In [9]:
def train_one_epoch():
    
    global X_data, Y_data
    
    for i in range(len(X_data)):
        inputs = X_data[i]
        labels = Y_data[i]
        
        optimizer.zero_grad()
        
        outputs = model(inputs)
        
        loss = loss_fn(outputs, labels)
        
        loss.backward()
        
        optimizer.step()
        
    return loss.item()

In [10]:
data()
model = TitanicModel()
loss_fn = torch.nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

In [11]:
X_data.type()

'torch.FloatTensor'

In [12]:
for epoch in range(EPOCHS):
    model.train(True)
    avg_loss = train_one_epoch()

In [13]:
sum_correct = 0
total_num = len(X_data)

In [14]:
for i in range(total_num):
    result = model(X_data[i])
    result = result.detach().numpy()
    result = 1 if result > 0.5 else 0
    
    if result == Y_data[i]:
        sum_correct += 1
print("Accuracy :", sum_correct/total_num)

Accuracy : 0.8058361391694725


In [15]:
submission_df = pd.DataFrame(columns=['PassengerId', 'Survived'])
submission_df['PassengerId'] = df_test['PassengerId']

In [16]:
# 예측 결과를 저장할 리스트 생성
survived_results = []

In [17]:
for i in range(len(X_test_data)):
    result = model(X_test_data[i])
    result = result.detach().numpy()
    result = 1 if result > 0.5 else 0
    
    # 예측 결과를 리스트에 저장
    survived_results.append(result)

# 예측 결과를 'Survived' 컬럼에 할당
submission_df['Survived'] = survived_results

In [18]:
submission_df.to_csv('submissions.csv', header=True, index=False)
print("Result saved")

Result saved


In [19]:
submission_df

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,0
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0
